In [1]:
import os, time, copy
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from random import seed, shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from skimage.color import gray2rgb
import matplotlib.pyplot as plt
from utils.myutils import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 设置TensorFlow的日志级别为2，只显示error和warining信息
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.get_logger().setLevel('ERROR') #删除多余的日志

import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import layers, models, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten,Reshape, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalMaxPool2D, Concatenate
from keras.optimizers import adam_v2
import os, time, copy
from glob import glob

from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Multiply
from tensorflow.keras.layers import Add
from tensorflow.keras.regularizers import l2,l1

import csv
from itertools import zip_longest

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
data_path = "./data/slices60/"  # 数据路径
model_path = "./model/VGG16/"  # 设置模型保存目录
# 创建模型目录
for path in [model_path]:
    print(path)
    if not os.path.exists(path):
        os.makedirs(path)

./model/VGG16/


# 1.数据预处理

#### 准备数据

In [3]:
# 数据
imagePaths = glob(data_path + "*/*flag_*.npy")  # 获取两个目录下的所有结节文件
imagePaths = np.asarray(imagePaths)  # 转numpy
print(imagePaths[:5])
print(f"共有{len(imagePaths)}个npy文件")
# 标签
labels = [x.split("flag_")[1].split(".npy")[0] for x in imagePaths]  # 提取各文件对应的标签(文件名中 "flag_"之后、".npy"之前的部分)
# print(labels[:5])
le = LabelEncoder()  # 标签编码器
labels = le.fit_transform(labels)  # 处理标签，标签转数字
print(f"标签类别: {le.classes_}")
print(labels[:5])

['./data/slices60/pos/nID_473, ID_622, X_133, Y_259, Z_189(186-211), diam_17, flag_1.npy'
 './data/slices60/pos/nID_418, ID_548, X_115, Y_286, Z_276(259-342), diam_46, flag_1.npy'
 './data/slices60/pos/nID_256, ID_353, X_112, Y_271, Z_235(228-248), diam_15, flag_1.npy'
 './data/slices60/pos/nID_256, ID_353, X_112, Y_271, Z_245(228-248), diam_15, flag_1.npy'
 './data/slices60/pos/nID_173, ID_243, X_419, Y_413, Z_264(226-268), diam_37, flag_1.npy']
共有10863个npy文件
标签类别: ['0' '1']
[1 1 1 1 1]


In [4]:
# 打乱顺序
seed(11)  # 设置random包的随机数种子
imageSets = list(zip(imagePaths, labels))  # 配对magePaths和labels中的元素
shuffle(imageSets)  #打乱元素顺序
print(imageSets[:5])
type(imageSets)
imageSets[0][0].split(" ID_")[1].split(",")[0]


[('./data/slices60/pos/nID_473, ID_622, X_133, Y_259, Z_189(186-211), diam_17, flag_1.npy', 1), ('./data/slices60/pos/nID_418, ID_548, X_115, Y_286, Z_276(259-342), diam_46, flag_1.npy', 1), ('./data/slices60/pos/nID_256, ID_353, X_112, Y_271, Z_235(228-248), diam_15, flag_1.npy', 1), ('./data/slices60/pos/nID_256, ID_353, X_112, Y_271, Z_245(228-248), diam_15, flag_1.npy', 1), ('./data/slices60/pos/nID_173, ID_243, X_419, Y_413, Z_264(226-268), diam_37, flag_1.npy', 1)]


'622'

#### 对病人按6：2：2划分训练集、验证集、测试集

In [5]:
for i in range(len(imageSets)):  # 遍历每张切片提取病人ID、结节nID、结节类型标签flag
    image_path, label = imageSets[i]
    ID = int(image_path.split(" ID_")[1].split(",")[0])
    nID = int(image_path.split("ID_")[1].split(",")[0])
    flag=int(image_path.split("flag_")[1].split(".npy")[0])
    imageSets[i] = (image_path, label, ID,nID,flag)


In [6]:
df = pd.DataFrame(imageSets, columns=['image_path', 'label', 'ID','nID','flag']) # 构建新数据框df

In [7]:
IDs = df["ID"].unique().tolist()  # 提取所有病人的ID
len(IDs)
data_list = df.values.tolist()  # df转化为二维列表
data_list[:5] 

[['./data/slices60/pos/nID_473, ID_622, X_133, Y_259, Z_189(186-211), diam_17, flag_1.npy',
  1,
  622,
  473,
  1],
 ['./data/slices60/pos/nID_418, ID_548, X_115, Y_286, Z_276(259-342), diam_46, flag_1.npy',
  1,
  548,
  418,
  1],
 ['./data/slices60/pos/nID_256, ID_353, X_112, Y_271, Z_235(228-248), diam_15, flag_1.npy',
  1,
  353,
  256,
  1],
 ['./data/slices60/pos/nID_256, ID_353, X_112, Y_271, Z_245(228-248), diam_15, flag_1.npy',
  1,
  353,
  256,
  1],
 ['./data/slices60/pos/nID_173, ID_243, X_419, Y_413, Z_264(226-268), diam_37, flag_1.npy',
  1,
  243,
  173,
  1]]

In [1]:
# 按病人划分训练集和测试集
def psplitDFbr(data, col_name, reset, test_size=0.2, seed=12, shuffle=True):
    # 全部病人ID列表
    IDs = data[col_name].unique().tolist()
    # 病人ID列表划分为训练集和测试集
    train_IDs, test_IDs = train_test_split(IDs, test_size=test_size, random_state=seed, shuffle=shuffle)
    # 得到两个数据集
    train_dat = data.loc[data[col_name].isin(train_IDs), ]  # 训练集
    test_dat = data.loc[data[col_name].isin(test_IDs),]  # 测试集
    if reset==True:  # 判断是在划分训练集和测试集，还是在划分训练集和验证集
        train_dat = train_dat.reset_index(drop=True)  # 训练集
        test_dat = test_dat.reset_index(drop=True)  # 测试集
    # 提取索引，用于模型训练
    train_index = train_dat.index.tolist()  # 训练集索引
    test_index = test_dat.index.tolist()  # 测试集索引
    # 返回训练集表、测试集表、训练集表索引、测试集表索引
    return train_dat, test_dat, train_index, test_index


In [10]:
# 按病人计算各数据子集中0-1分类占比，观察样本平衡性
def pcalRatebr(data, index):
    # 是否通过索引进行筛选
    if index is None:
        n = data.shape[0]  # 如果没有提供索引，使用整个数据集的行数作为样本量
    else:
        n = len(index)  # 如果提供了索引，使用索引的长度作为样本量
        data = data.loc[index, ]   # 根据索引筛选数据
    
    nn = len(data["nID"].unique()) # 结节数量
    pp=  len(data["ID"].unique())  #病人数量
    rate1 = round(data["flag"].sum() / n * 100, 2)  # 计算数据子集中微乳头切片占比
    rate2 = round(data.groupby("ID")["flag"].mean().mean() * 100, 2)  # 计算数据子集中微乳头结节占比
    return f"切片数: {n}，微乳头切片占比: {rate1}%，结节数: {nn}, 微乳头结节占比：{rate2}%，病人数: {pp}"

In [11]:
train_data, test_dat, _, test_index = psplitDFbr(df, "ID", test_size=0.2, reset=True)  # 按病人ID划分为训练集和测试集
train_dat, valid_dat, train_index, valid_index = psplitDFbr(train_data, "ID", test_size=0.25, reset=False)  # 按病人ID将训练集再划分为训练集和验证集
print(len(train_dat["ID"].unique().tolist()))
len(train_dat)
print(f"训练数据中训练集{pcalRatebr(train_dat, train_index)}")
print(f"训练数据中验证集{pcalRatebr(valid_dat, valid_index)}")
print(f"测试数据中测试集{pcalRatebr(test_dat, test_index)}")

255
训练数据中训练集切片数: 6578，微乳头切片占比: 38.81%，结节数: 291, 微乳头结节占比：31.2%，病人数: 255
训练数据中验证集切片数: 2278，微乳头切片占比: 34.77%，结节数: 99, 微乳头结节占比：31.0%，病人数: 85
测试数据中测试集切片数: 2007，微乳头切片占比: 40.01%，结节数: 93, 微乳头结节占比：28.35%，病人数: 86


#### 记录数据集按病人的划分情况

In [ ]:
merged_list = list(zip_longest(train_IDs,valid_IDs,test_IDs)) #合并三列ID成一个元组的列表，并用默认值填充，确保所有元组长度相同
merged_list_name=['train_IDs','valid_IDs','test_IDs']
data_dict_list = [dict(zip(merged_list_name, row)) for row in merged_list] #创建一个包含字典的列表

In [ ]:
# 将包含训练集、验证集和测试集病人ID的数据字典列表写入到名为'output.csv'的CSV文件中
output_file = 'output.csv' 
with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['train_IDs','valid_IDs','test_IDs']  # 列名
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    # 写入列名
    csv_writer.writeheader()
    # 写入数据
    csv_writer.writerows(data_dict_list)

In [12]:
# 数据提取为列表形式
# train_dat.values.tolist() #训练集数据转化为列表
# 从训练集中提取除了前两列（image_path和label）之外的所有列的数据，将其转换为列表
trainSets=train_dat.drop(columns=df.columns[2:]).values.tolist()
# 将训练集的每一行数据转换为元组，并存储在trainSets列表中
trainSets = [tuple(data) for data in trainSets]
testSets=test_dat.drop(columns=df.columns[2:]).values.tolist()
testSets = [tuple(data) for data in testSets]
validSets=valid_dat.drop(columns=df.columns[2:]).values.tolist()
validSets = [tuple(data) for data in validSets]


In [13]:
# 将数据中的每一行数据转换为元组
for i in range(len(trainSets)):
    trainSets[i] = tuple(trainSets[i])  
type(trainSets)
for i in range(len(testSets)):
    testSets[i] = tuple(testSets[i]) 
for i in range(len(validSets)):
    validSets[i] = tuple(validSets[i]) 

#### 数据生成器

In [16]:
# 设置参数
trainbatchsize = 60  # 训练集batch
valbatchsize = 40  # 测试集batch
epochs = 50  # 训练epoch

# 数据生成器
train_generator = data_generator(trainset, batch_size=trainbatchsize, n_labels=1, labels=None, shuffle_index_list=True)
val_generator = data_generator(validSets, batch_size=valbatchsize, n_labels=1, labels=None, shuffle_index_list=True)

# 计算训练和验证的步数，在训练的时候需要这两个参数
train_steps = get_number_of_steps(len(trainset), trainbatchsize)
val_steps = get_number_of_steps(len(validSets), valbatchsize)

# 2.特征提取模型

In [17]:
# 记录训练开始时间，同时作为保存模型文件名的前缀
train_start = time.strftime("%Y-%m-%d %H:%M:%S")
# 初始学习率为0.001，如果20个epoch后精度没有提升，学习率降低0.5；存外样本精度最高的模型
# 构建模型文件名，包括训练开始时间（train_start），模型类型（VGG16_cbam2fin08_best.h5）
model_file = os.path.join(model_path, f'{train_start} VGG16_cbam2fin08_best.h5')
# 构建日志文件名，包括训练开始时间（train_start）和模型类型（VGG16_cbam08fin_metric.csv）
logging_file = os.path.join(model_path, f"{train_start} VGG16_cbam08fin_metric.csv")
# 获取回调函数，用于在训练时进行学习率衰减、保存最佳模型、记录日志等
callback = get_callbacks(model_file, initial_learning_rate=0.001, learning_rate_drop=0.5, learning_rate_epochs=None,
                  learning_rate_patience=20, logging_file=logging_file, verbosity=1,
                  early_stopping_patience=10,tensorboard_callback = None)

#### 迁移学习VGG16模型，在第二个模块后加入CBAM注意力机制（消融实验所采用的backbone附在该代码的最后）

In [19]:
#  VGG16+CBAM(消融实验部分仅需要把该模块backbone替换掉即可)
# 定义CBAM模块
def cbam_module(x):
    # 全局平均池化和全局最大池化
    avgpool = GlobalAveragePooling2D(name='channel_avgpool')(x)
    maxpool = GlobalMaxPool2D(name='channel_maxpool')(x)
    # Shared MLP
    Dense_layer1 = Dense(128//8,activation='relu', name='channel_fc1')
    Dense_layer2 = Dense(128, activation='relu', name='channel_fc2')
    avg_out = Dense_layer2(Dense_layer1(avgpool))
    max_out = Dense_layer2(Dense_layer1(maxpool))

    ## Channel Attention
    channel = layers.add([avg_out, max_out])
    channel = Activation('sigmoid', name='channel_sigmoid')(channel)
    channel = Reshape((1, 1, 128), name='channel_reshape')(channel)
    channel = Conv2D(128, (1, 1), strides=(1, 1), padding='same', name='adjust_channel_dim')(channel)
    channel_out = tf.multiply(x, channel)
    
    # Spatial Attention
    avgpool = tf.reduce_mean(channel_out, axis=3, keepdims=True, name='spatial_avgpool')
    maxpool = tf.reduce_max(channel_out, axis=3, keepdims=True, name='spatial_maxpool')
    spatial = Concatenate(axis=3)([avgpool, maxpool])

    spatial = Conv2D(1, (7,7), strides=1, padding='same',name='spatial_conv2d')(spatial)
    spatial_out = Activation('sigmoid', name='spatial_sigmoid')(spatial)
    #spatial_out=BatchNormalization(x)

    #CBAM输出
    CBAM_out = tf.multiply(channel_out, spatial_out)
    x=CBAM_out
    return x
# 实例化一个具有预训练权重的基础模型
base_model = VGG16(weights="imagenet", input_shape=(60,60,3), include_top=False)
# 冻结该基础模型，参数不可变
base_model.trainable = False
# 根据基础模型创建一个新模型
inputs = keras.Input(shape=(60, 60, 3))# 输入
x = base_model(inputs, training=False)
# 获取VGG16模型的前2个blocks
blocks_1_2 = base_model.layers[1:6]
blocks_3_5=base_model.layers[7:19]
# 创建新的VGG16模型，并在第2个block后插入CBAM模块
x = inputs
for layer in blocks_1_2:
    x = layer(x)
    if layer.name == 'block2_conv2':
        x = cbam_module(x)
# 第3-5个block仍为原迁移学习模型
for layer in blocks_3_5:
    x = layer(x)
# 增加新层
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
#创建一个Dropout层
x=layers.Dropout(0.5)(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1,kernel_regularizer=l2(0.01),activity_regularizer=l1(0.02))(x)
print(outputs.shape)
# 创建新模型
model_cbam = keras.Model(inputs, outputs)
model_cbam.summary()

(?, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 60, 60, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 60, 60, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 60, 60, 64)   36928       block1_conv1[2][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 30, 30, 64)   0           block1_conv2[2][0]               
_______________________________________________________________________________________

In [21]:
#测量代码执行时间
%%time
# 设置模型 模型编译
model_cbam.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
# 训练模型
model_cbam.fit(train_generator, steps_per_epoch=train_steps, epochs=50, callbacks=callback, 
          validation_data=val_generator, validation_steps=val_steps)

Epoch 1/50
144/144 [==============================] - ETA: 0s - batch: 71.5000 - size: 59.6181 - loss: 0.6680 - binary_accuracy: 0.6260

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/root/miniconda3/envs/myconda/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


144/144 [==============================] - 55s 322ms/step - batch: 71.5000 - size: 59.6181 - loss: 0.6680 - binary_accuracy: 0.6260 - val_loss: 0.6407 - val_binary_accuracy: 0.6941 - lr: 0.0010
Epoch 2/50
144/144 [==============================] - 38s 261ms/step - batch: 71.5000 - size: 59.6181 - loss: 0.5964 - binary_accuracy: 0.6683 - val_loss: 0.5539 - val_binary_accuracy: 0.6791 - lr: 0.0010
Epoch 3/50
144/144 [==============================] - 40s 277ms/step - batch: 71.5000 - size: 59.6181 - loss: 0.5717 - binary_accuracy: 0.6990 - val_loss: 0.5428 - val_binary_accuracy: 0.7155 - lr: 0.0010
Epoch 4/50
144/144 [==============================] - 39s 273ms/step - batch: 71.5000 - size: 59.6181 - loss: 0.5637 - binary_accuracy: 0.7005 - val_loss: 0.5359 - val_binary_accuracy: 0.7424 - lr: 0.0010
Epoch 5/50
144/144 [==============================] - 38s 261ms/step - batch: 71.5000 - size: 59.6181 - loss: 0.5515 - binary_accuracy: 0.7117 - val_loss: 0.5288 - val_binary_accuracy: 0.7304

# 消融实验的backbone

#### VGG16+Channel Attention

In [ ]:
# 定义CA模块
def CA_module(x):
    avgpool = GlobalAveragePooling2D(name='channel_avgpool')(x)
    maxpool = GlobalMaxPool2D(name='channel_maxpool')(x)
    # Shared MLP
    Dense_layer1 = Dense(128//8,activation='relu', name='channel_fc1')
    Dense_layer2 = Dense(128, activation='relu', name='channel_fc2')
    avg_out = Dense_layer2(Dense_layer1(avgpool))
    max_out = Dense_layer2(Dense_layer1(maxpool))

    #Channel Attention
    channel = layers.add([avg_out, max_out])
    channel = Activation('sigmoid', name='channel_sigmoid')(channel)
    channel = Reshape((1, 1, 128), name='channel_reshape')(channel)
    channel = Conv2D(128, (1, 1), strides=(1, 1), padding='same', name='adjust_channel_dim')(channel)
    #channel = Reshape((15,15,256), name='channel_reshape')(channel)
    channel_out = tf.multiply(x, channel)

    x=channel_out
    return x
# 实例化一个具有预训练权重的基础模型
base_model = VGG16(weights="imagenet", input_shape=(60,60,3), include_top=False)
# 冻结该基础模型，参数不可变
base_model.trainable = False
# 根据基础模型创建一个新模型
inputs = keras.Input(shape=(60, 60, 3))# 输入
x = base_model(inputs, training=False)
# 获取VGG16模型的前2个blocks
blocks_1_2 = base_model.layers[1:6]
blocks_3_5=base_model.layers[7:19]
# 创建新的VGG16模型，并在第2个block后插入CBAM模块
x = inputs
for layer in blocks_1_2:
    x = layer(x)
    if layer.name == 'block2_conv2':
        x = CA_module(x)
# 第3-5个block仍为原迁移学习模型
for layer in blocks_3_5:
    x = layer(x)
# 增加新层
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
#创建一个Dropout层
x=layers.Dropout(0.5)(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1,kernel_regularizer=l2(0.01),activity_regularizer=l1(0.02))(x)
print(outputs.shape)
# 创建新模型
model_CA = keras.Model(inputs, outputs)
model_CA.summary()

####  VGG16+Spatial Attention

In [17]:
# 定义SA模块
def SA_module(x): 
# Spatial Attention
    avgpool = tf.reduce_mean(x, axis=3, keepdims=True, name='spatial_avgpool')
    maxpool = tf.reduce_max(x, axis=3, keepdims=True, name='spatial_maxpool')
    spatial = Concatenate(axis=3)([avgpool, maxpool])

    spatial = Conv2D(1, (7,7), strides=1, padding='same',name='spatial_conv2d')(spatial)
    spatial_out = Activation('sigmoid', name='spatial_sigmoid')(spatial)
    spatial_out = BatchNormalization()(spatial_out)
    spatial_out = Conv2D(256, (1, 1), strides=1, padding='same', name='spatial_conv2d_dim')(spatial_out)
    
    x=spatial_out
    return x
# 实例化一个具有预训练权重的基础模型
base_model = VGG16(weights="imagenet", input_shape=(60,60,3), include_top=False)
# 冻结该基础模型，参数不可变
base_model.trainable = False
# 根据基础模型创建一个新模型
inputs = keras.Input(shape=(60, 60, 3))# 输入
x = base_model(inputs, training=False)
# 获取VGG16模型的前2个blocks
blocks_1_2 = base_model.layers[1:6]
blocks_3_5=base_model.layers[7:19]
# 创建新的VGG16模型，并在第2个block后插入CBAM模块
x = inputs
for layer in blocks_1_2:
    x = layer(x)
    if layer.name == 'block2_conv2':
        x = SA_module(x)
# 第3-5个block仍为原迁移学习模型
for layer in blocks_3_5:
    x = layer(x)
# 增加新层
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
#创建一个Dropout层
x=layers.Dropout(0.5)(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1,kernel_regularizer=l2(0.01),activity_regularizer=l1(0.02))(x)
print(outputs.shape)
# 创建新模型
model_SA = keras.Model(inputs, outputs)
model_SA.summary()

(?, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 60, 60, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 60, 60, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 60, 60, 64)   36928       block1_conv1[2][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 30, 30, 64)   0           block1_conv2[2][0]               
_______________________________________________________________________________________

#### VGG16

In [20]:
##VGG16
# 实例化一个具有预训练权重的基础模型
base_model = VGG16(weights="imagenet", input_shape=(60,60,3), include_top=False)
# 冻结该基础模型，参数不可变
base_model.trainable = False
# 根据基础模型创建一个新模型
# 输入
inputs = keras.Input(shape=(60, 60, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.

x = base_model(inputs, training=False)

# 增加新层
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
print(outputs.shape)
# 创建新模型
model = keras.Model(inputs, outputs)
model.summary()

(?, 1)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 60, 60, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
